In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import dblquad
from scipy.integrate import quad


In [1]:
import profiles

c:\python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
# Function to calculate the NFW halo density profile
def nfw_halo(r, rs,rhos):
    x = r / rs
    return rhos / (x * (1 + x)**2)


In [75]:


# Function to calculate the Milky Way density profile based on Gondolo-Silk model
def mw_density_GS(r,rs,rhos,Rs, Rsat, Rsp, gamma_sp, rho_sat):
    
    density = np.zeros_like(r)

    # Condition 1: r < 2RS
    mask_condition_1 = r < 2 * Rs
    density[mask_condition_1] = 0

    # Condition 2: 2RS ≤ r < Rsat
    mask_condition_2 = (r >= 2 * Rs) & (r < Rsat)
    density[mask_condition_2] = rho_sat * (r[mask_condition_2] / Rsat)**(-0.5)

    # Condition 3: Rsat ≤ r < Rsp
    mask_condition_3 = (r >= Rsat) & (r < Rsp)
    x = r[mask_condition_3] / Rsp
    density[mask_condition_3] = nfw_halo(Rsp,rs,rhos) * x**(-gamma_sp)

    # Condition 4: r ≥ Rsp
    mask_condition_4 = r >= Rsp
    density[mask_condition_4] = nfw_halo(r[mask_condition_4],rs,rhos)

    return density



In [140]:


# Constants and parameters
rs = 18.6  # Scale radius in kpc
rho0 = 0.383  # Local DM density in GeV/cm^3
r0 = 8.2  # Sun position in kpc
m_chi = 1000# Dark matter mass in GeV
t = 3.15 * 10**17  # in s
rhos = (rho0 * ((r0 / rs) * (1 + r0 / rs)**(2)))
R = 200
# Parameters for the Milky Way density profile
Rs = 3.8 * 10**(-11)  # Example value for Rs
Rsp = 0.34 * 10**(-3)  # Example value for Rsp
gamma_sp = 2.35  # Example value for gamma_sp



In [148]:

integrand = lambda l: (nfw_halo(np.abs(r0-l),rs,rhos)**2)*3.086*10**21*0.1
J_halo,error = quad(integrand,0,r0+R)

C:\Users\RITABIK BANERJEE\AppData\Local\Temp\ipykernel_31688\3137348193.py:2: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  J_halo,error = quad(integrand,0,r0+R)


In [149]:
J_halo

3.086741529207028e+27

In [145]:
integrand = lambda l: (mw_density_GS(np.abs(r0-l),rs,rhos,Rs,0,Rsp,gamma_sp,np.inf)**2)*l**2
J_GS,error = quad(integrand,0,R+r0)

C:\Users\RITABIK BANERJEE\AppData\Local\Temp\ipykernel_31688\1579548711.py:2: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  J_GS,error = quad(integrand,0,R+r0)


In [144]:
J_GS

0.011928588652407257